In [28]:
import pandas as pd # type: ignore
from datetime import datetime # type: ignore

In [29]:
import glob

folder_path = 'datasets/LJP'

# Get a list of all CSV files in the folder
csv_files = glob.glob(folder_path + '/*.csv')

# Create an empty list to store the DataFrames
dfs = []

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
	df = pd.read_csv(file)
	dfs.append(df)

# Concatenate all the DataFrames together
df_arso = pd.concat(dfs, ignore_index=True)
df_arso.sort_values(by=' valid')
df_arso

,station id,station name,valid,T [°C],količina padavin [mm],globalno sev. [W/m2]
0,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:00,-2.6,0.0,0.0
1,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:10,-2.6,NaN,NaN
2,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:20,-2.6,NaN,NaN
3,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:30,-3.4,0.0,0.0
4,_3047,Letališče Jožeta Pučnika Ljubljana,2022-01-01 00:40,-2.9,NaN,NaN
...,...,...,...,...,...,...
152923,_3047,Letališče Jožeta Pučnika Ljubljana,2023-01-31 23:10,-5.4,NaN,NaN
152924,_3047,Letališče Jožeta Pučnika Ljubljana,2023-01-31 23:20,-4.7,NaN,NaN
152925,_3047,Letališče Jožeta Pučnika Ljubljana,2023-01-31 23:30,-5.4,0.0,0.0
152926,_3047,Letališče Jožeta Pučnika Ljubljana,2023-01-31 23:40,-5.9,NaN,NaN


In [30]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_arso):
	# Filter rows based on column: ' valid'
	df_arso = df_arso[df_arso[' valid'].str.endswith("00", na=False)]
	# Drop columns: 'station id', ' station name'
	df_arso = df_arso.drop(columns=['station id', ' station name'])
	# Change column type to datetime64[ns] for column: ' valid'
	df_arso = df_arso.astype({' valid': 'datetime64[ns]'})
	# Rename column ' valid' to 'datum'
	df_arso = df_arso.rename(columns={' valid': 'datum'})
	df_arso.sort_values(by='datum', inplace=True)
	return df_arso

df_arso_clean = clean_data(df_arso.copy())
df_arso_clean

,datum,T [°C],količina padavin [mm],globalno sev. [W/m2]
0,2022-01-01 00:00:00,-2.6,0.0,0.0
6,2022-01-01 01:00:00,-3.2,0.0,0.0
12,2022-01-01 02:00:00,-3.9,0.0,0.0
18,2022-01-01 03:00:00,-2.8,0.0,0.0
24,2022-01-01 04:00:00,-3.1,0.0,0.0
...,...,...,...,...
117618,2024-10-28 19:00:00,14.1,0.0,0.0
117624,2024-10-28 20:00:00,14.1,0.0,0.0
117630,2024-10-28 21:00:00,14.0,0.0,0.0
117636,2024-10-28 22:00:00,13.9,0.0,0.0


In [31]:
df_eles2 = pd.read_csv('datasets/main/ELES2.csv', sep=',')
df_eles2.rename(columns={'Unnamed: 0.1': 'datum', 'PREDVIDEN PREVZEM': 'predviden_prevzem', 'DEJANSKI PREVZEM': 'dejanski_prevzem'}, inplace=True)
df_eles2['datum'] = pd.to_datetime(df_eles2['datum'])
df_eles2.drop(columns=['Unnamed: 0', 'PREDVIDENA PROIZVODNJA', 'DEJANSKA PROIZVODNJA'], inplace=True)
df_eles2

,datum,predviden_prevzem,dejanski_prevzem
0,2022-01-01 00:00:00,989.0,1044.0
1,2022-01-01 01:00:00,953.0,1018.0
2,2022-01-01 02:00:00,906.0,975.0
3,2022-01-01 03:00:00,874.0,944.0
4,2022-01-01 04:00:00,870.0,936.0
...,...,...,...
24931,2024-11-03 19:00:00,1354.0,1304.0
24932,2024-11-03 20:00:00,1309.0,1262.0
24933,2024-11-03 21:00:00,1212.0,NaN
24934,2024-11-03 22:00:00,1131.0,NaN


In [32]:
df_eles = pd.read_csv('datasets/main/ELES.csv', sep=';')

"""
Cell generated by Data Wrangler.
"""
from datetime import datetime

def clean_data(df_eles):
    df_eles['datum'] = pd.to_datetime(df_eles['datum'], dayfirst=True)
    df_eles['ura'] = df_eles['ura'].str[1:3].astype(int) - 1
    df_eles['datum'] = df_eles['datum'] + pd.to_timedelta(df_eles['ura'], unit='h')
    df_eles.drop(columns=['ura', 'predvidena_proizvodnja', 'dejanska_proizvodnja', 'nek', 'termo', 'hidro'], inplace=True)
    # Filter rows based on column: 'datum'
    # df_eles = df_eles[df_eles['datum'] >= datetime.strptime('2024-08-01T00:00:00.000Z', '%Y-%m-%dT%H:%M:%S.%fZ')]
    return df_eles

df_eles_clean = clean_data(df_eles.copy())
df_eles_clean

,datum,predviden_prevzem,dejanski_prevzem
0,2022-01-01 00:00:00,989,1044.0
1,2022-01-01 01:00:00,953,1018.0
2,2022-01-01 02:00:00,906,975.0
3,2022-01-01 03:00:00,874,944.0
4,2022-01-01 04:00:00,870,936.0
...,...,...,...
24904,2024-11-03 19:00:00,2700,NaN
24905,2024-11-03 20:00:00,2611,NaN
24906,2024-11-03 21:00:00,2415,NaN
24907,2024-11-03 22:00:00,2254,NaN


In [33]:
df_pokritost = pd.read_csv('datasets/main/LJP_pokritost_2022-2024.csv')
df_pokritost['datum'] = pd.to_datetime(df_pokritost[' valid'])
df_pokritost = df_pokritost.drop(columns=['station id', ' station name', ' valid'])
df_pokritost

,oblačnost [%],datum
0,37.0,2022-01-01
1,40.0,2022-01-02
2,80.0,2022-01-03
3,100.0,2022-01-04
4,100.0,2022-01-05
...,...,...
1027,100.0,2024-10-24
1028,100.0,2024-10-25
1029,73.0,2024-10-26
1030,27.0,2024-10-27


In [34]:
from prophet.make_holidays import make_holidays_df # type: ignore

df_prazniki = make_holidays_df(
    year_list=[2022 + i for i in range(3)], country='SI'
)
df_prazniki = df_prazniki.rename(columns={'ds': 'datum'})
df_prazniki.sort_values('datum', inplace=True)
df_prazniki['datum'] = pd.to_datetime(df_prazniki['datum'])

# Generate a new DataFrame with all dates between 2022-01-01 and 2024-10-23
df_all_dates = pd.DataFrame({'datum': pd.date_range('2022-01-01', '2024-12-31')})

# Merge the new DataFrame with df_prazniki, filling in missing values with 0
df_prazniki = pd.merge(df_all_dates, df_prazniki, on='datum', how='outer').fillna(0)

df_prazniki

,datum,holiday
0,2022-01-01,New Year's Day
1,2022-01-02,New Year's Day
2,2022-01-03,0
3,2022-01-04,0
4,2022-01-05,0
...,...,...
1091,2024-12-27,0
1092,2024-12-28,0
1093,2024-12-29,0
1094,2024-12-30,0


In [35]:
df_prices = pd.read_csv('datasets/main/day_ahead_prices.csv', index_col=0)

# drop if there is a missing value in index
df_prices = df_prices[df_prices.index.notna()]

df_prices['datum'] = df_prices.index
df_prices['datum'] = pd.to_datetime(df_prices['datum']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))

df_prices.sort_values(by='datum', inplace=True)
df_prices.reset_index(drop=True, inplace=True)
df_prices.drop(columns='Hour_q', inplace=True)

df_prices.rename(columns={'Price': 'price'}, inplace=True)

df_prices

C:\Users\patri\AppData\Local\Temp\ipykernel_7148\565848784.py:7: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_prices['datum'] = pd.to_datetime(df_prices['datum']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))


,price,datum
0,61.83783,2022-01-01 00:00:00
1,51.44327,2022-01-01 01:00:00
2,55.91396,2022-01-01 02:00:00
3,46.94279,2022-01-01 03:00:00
4,37.67210,2022-01-01 04:00:00
...,...,...
26293,NaN,2024-12-31 19:00:00
26294,NaN,2024-12-31 20:00:00
26295,NaN,2024-12-31 21:00:00
26296,NaN,2024-12-31 22:00:00


In [36]:
df_volumes = pd.read_csv('datasets/main/day_ahead_volumes.csv', index_col=0)

# drop if there is a missing value in index
df_volumes = df_volumes[df_volumes.index.notna()]

df_volumes['datum'] = df_volumes.index
df_volumes['datum'] = pd.to_datetime(df_volumes['datum']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))

df_volumes.sort_values(by='datum', inplace=True)
df_volumes.reset_index(drop=True, inplace=True)
df_volumes.drop(columns='Hour_q', inplace=True)
df_volumes.rename(columns={'Price': 'volumes'}, inplace=True)

df_volumes

C:\Users\patri\AppData\Local\Temp\ipykernel_7148\771561581.py:7: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_volumes['datum'] = pd.to_datetime(df_volumes['datum']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))


,volumes,datum
0,942.200,2022-01-01 00:00:00
1,1084.700,2022-01-01 01:00:00
2,1071.100,2022-01-01 02:00:00
3,1065.700,2022-01-01 03:00:00
4,1062.800,2022-01-01 04:00:00
...,...,...
24875,1629.500,2024-11-04 19:00:00
24876,1561.226,2024-11-04 20:00:00
24877,1606.500,2024-11-04 21:00:00
24878,1445.700,2024-11-04 22:00:00


In [37]:
df_prices_volumes = pd.merge(df_prices, df_volumes, on='datum', how='left')
df_prices_volumes

,price,datum,volumes
0,61.83783,2022-01-01 00:00:00,942.2
1,51.44327,2022-01-01 01:00:00,1084.7
2,55.91396,2022-01-01 02:00:00,1071.1
3,46.94279,2022-01-01 03:00:00,1065.7
4,37.67210,2022-01-01 04:00:00,1062.8
...,...,...,...
26293,NaN,2024-12-31 19:00:00,NaN
26294,NaN,2024-12-31 20:00:00,NaN
26295,NaN,2024-12-31 21:00:00,NaN
26296,NaN,2024-12-31 22:00:00,NaN


In [38]:
df1 = pd.merge(df_eles2, df_arso_clean, on='datum', how='left')

df1['datum'] = pd.to_datetime(df1['datum']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
df_prices_volumes['datum'] = pd.to_datetime(df_prices_volumes['datum']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))

df2 = pd.merge(df1, df_prices_volumes, on='datum', how='right')
df2

,datum,predviden_prevzem,dejanski_prevzem,T [°C],količina padavin [mm],globalno sev. [W/m2],price,volumes
0,2022-01-01 00:00:00,989.0,1044.0,-2.6,0.0,0.0,61.83783,942.2
1,2022-01-01 01:00:00,953.0,1018.0,-3.2,0.0,0.0,51.44327,1084.7
2,2022-01-01 02:00:00,906.0,975.0,-3.9,0.0,0.0,55.91396,1071.1
3,2022-01-01 03:00:00,874.0,944.0,-2.8,0.0,0.0,46.94279,1065.7
4,2022-01-01 04:00:00,870.0,936.0,-3.1,0.0,0.0,37.67210,1062.8
...,...,...,...,...,...,...,...,...
27037,2024-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27038,2024-12-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27039,2024-12-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27040,2024-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df3 = pd.merge(df_pokritost, df_prazniki, on='datum', how='right')
df3

,oblačnost [%],datum,holiday
0,37.0,2022-01-01,New Year's Day
1,40.0,2022-01-02,New Year's Day
2,80.0,2022-01-03,0
3,100.0,2022-01-04,0
4,100.0,2022-01-05,0
...,...,...,...
1091,NaN,2024-12-27,0
1092,NaN,2024-12-28,0
1093,NaN,2024-12-29,0
1094,NaN,2024-12-30,0


In [40]:
df2['datum'] = pd.to_datetime(df2['datum']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
df3['datum'] = pd.to_datetime(df3['datum']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))

df = pd.merge(df2, df3, on='datum', how='left')

df

,datum,predviden_prevzem,dejanski_prevzem,T [°C],količina padavin [mm],globalno sev. [W/m2],price,volumes,oblačnost [%],holiday
0,2022-01-01 00:00:00,989.0,1044.0,-2.6,0.0,0.0,61.83783,942.2,37.0,New Year's Day
1,2022-01-01 01:00:00,953.0,1018.0,-3.2,0.0,0.0,51.44327,1084.7,NaN,NaN
2,2022-01-01 02:00:00,906.0,975.0,-3.9,0.0,0.0,55.91396,1071.1,NaN,NaN
3,2022-01-01 03:00:00,874.0,944.0,-2.8,0.0,0.0,46.94279,1065.7,NaN,NaN
4,2022-01-01 04:00:00,870.0,936.0,-3.1,0.0,0.0,37.67210,1062.8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
27037,2024-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27038,2024-12-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27039,2024-12-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27040,2024-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
gas_prices = pd.read_csv('datasets/main/Slovenia.csv')
gas_prices.drop(columns=['Country', 'ISO3 Code', 'Datetime (UTC)'], inplace=True)
gas_prices.rename(columns={'Datetime (Local)': 'date'}, inplace=True)
gas_prices['date'] = gas_prices['date'].astype('datetime64[ns]')
# pandas filter by datetime (local) column
gas_prices = gas_prices[gas_prices['date'] >= '2022-01-01 00:00:00']
# test_data = df_hourly[df_hourly['date'] >= cutoff_date]
gas_prices #unused

,date,Price (EUR/MWhe)
61367,2022-01-01 00:00:00,61.84
61368,2022-01-01 01:00:00,51.44
61369,2022-01-01 02:00:00,55.91
61370,2022-01-01 03:00:00,46.94
61371,2022-01-01 04:00:00,37.67
...,...,...
85459,2024-09-30 21:00:00,49.27
85460,2024-09-30 22:00:00,45.20
85461,2024-09-30 23:00:00,9.84
85462,2024-10-01 00:00:00,3.21


In [42]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
	# Drop duplicate rows in column: 'datum'
	df = df.drop_duplicates(subset=['datum'])
	# Renames
	df = df.rename(columns={'datum': 'date', 
							'dejanski_prevzem': 'consumption',
							'T [°C]': 'temperature',
							'količina padavin [mm]': 'precipitation',
							'globalno sev. [W/m2]': 'irradiation',
							'predviden_prevzem': 'consumption_eles_forecast',
							'oblačnost [%]': 'cloudiness'
						})
	df.ffill(inplace=True)
	
	df['date'] = df['date'].astype('datetime64[ns]')
	df = df[df['date'] < '2024-11-05 00:00:00']

	df = df[(df['date'].dt.day != 29) | (df['date'].dt.month != 2)]
	df = df.drop(df.index[0])

	return df


df = clean_data(df.copy())
df

,date,consumption_eles_forecast,consumption,temperature,precipitation,irradiation,price,volumes,cloudiness,holiday
1,2022-01-01 01:00:00,953.0,1018.0,-3.2,0.0,0.0,51.44327,1084.700,37.0,New Year's Day
2,2022-01-01 02:00:00,906.0,975.0,-3.9,0.0,0.0,55.91396,1071.100,37.0,New Year's Day
3,2022-01-01 03:00:00,874.0,944.0,-2.8,0.0,0.0,46.94279,1065.700,37.0,New Year's Day
4,2022-01-01 04:00:00,870.0,936.0,-3.1,0.0,0.0,37.67210,1062.800,37.0,New Year's Day
5,2022-01-01 05:00:00,892.0,952.0,-4.1,0.0,0.0,39.70000,1070.000,37.0,New Year's Day
...,...,...,...,...,...,...,...,...,...,...
25669,2024-11-04 19:00:00,1045.0,1262.0,13.8,0.0,0.0,183.82000,1629.500,83.0,0
25670,2024-11-04 20:00:00,1045.0,1262.0,13.8,0.0,0.0,143.15000,1561.226,83.0,0
25671,2024-11-04 21:00:00,1045.0,1262.0,13.8,0.0,0.0,118.17000,1606.500,83.0,0
25672,2024-11-04 22:00:00,1045.0,1262.0,13.8,0.0,0.0,108.33000,1445.700,83.0,0


In [43]:
df.to_csv('datasets/dataset_PS.csv', index=False)